# Efficiency - Delete multiple points

| comparable algo list |
| - |
| delta |
| Monte Carlo |

In [ ]:
import sys
sys.path.append("..")

from sklearn import svm
import numpy as np

import dynashap

from examples.data_utils import (
    load_npy, save_npy, preprocess_data, variance, normalize
)

In [ ]:
model = svm.SVC(decision_function_shape='ovo')

size_choices = [10, 100, 1000, 10000]
m_choices = [10, 15, 18, 20, 22, 25, 28, 30]
stone_m = 100

# for 200 points
size_choices = [200]
m_choices = [1, 2, 3, 5, 8]
mc_stone = 30

for size in size_choices:
    x_train, y_train, x_test, y_test, \
    columns_name = preprocess_data(
        't_train_' + str(size) + 'p.csv',
        't_test_' + str(size) + 'p.csv')

    # process data
    x_train_ = np.append(x_train, x_test[:2], axis=0)
    y_train_ = np.append(y_train, y_test[:2])
    x_test_ = x_test[2:, :]
    y_test_ = y_test[2:]

    # get a init sv
    init_sv = dynashap.mc_shap(x_train_, y_train_, x_test_, y_test_, model, stone_m * len(y_train_), proc_num=20)
    save_npy('dm_init_mc_plus_size' + str(size) + '.npy', init_sv)

    # set a stone
    mc_plus = dynashap.mc_shap(x_train, y_train, x_test_, y_test_, model, stone_m * len(y_train), proc_num=20)
    save_npy('dm_mc_plus_size' + str(size) + '.npy', mc_plus)

    # mc
    mc = dynashap.mc_shap(x_train, y_train, x_test_, y_test_, model, mc_stone * len(y_train), proc_num=20)
    save_npy('dm_mc_size'+ str(size) +'m' + str(mc_stone) + '.npy', mc)

    # # walk the m choices
    for m in m_choices:
       # delta
        delta_shap = dynashap.DeltaShap(x_train_, y_train_, x_test_, y_test_, model,
                                        init_sv)
        m_delta_sv = delta_shap.del_single_point(len(y_train_) - 1, m=(len(y_train_)-1)*m, proc_num=20)

        n_delta_shap = dynashap.DeltaShap(x_train_[:-1,:], y_train_[:-1], x_test_,
                                        y_test_, model, m_delta_sv)
        delta_sv = n_delta_shap.del_single_point(len(y_train_) - 2, (len(y_train_)-2)*m, proc_num=20)
        save_npy('dm_delta_size' + str(size) +'m' + str(m) + '.npy', delta_sv)